In [26]:
import ast

import pandas as pd
import janitor
import altair as alt
from pydash import py_

from funcs.paths import paths
print(paths)

{'root': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm'), 'data_root': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/data'), 'raw_data_dir': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/data/local-source-data'), 'examples_data_dir': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/data/local-source-data/examples'), 'output': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/output'), 'results': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/output/results'), 'artifacts': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/output/artifacts'), 'tmp_output': PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/output/tmp')}


In [2]:
INPUT_DIR = paths["results"] / "metadata_extract"
assert INPUT_DIR.exists(), INPUT_DIR
OUTPUT_DIR = paths["results"]
assert OUTPUT_DIR.exists(), OUTPUT_DIR

----

# Collect mass extract results

In [15]:
input_file = INPUT_DIR / "meta_info.csv"
assert input_file.exists(), input_file

meta_info = pd.read_csv(input_file)
meta_info

,input_dir,output_results_file,output_error_file
0,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
1,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
2,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
3,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
4,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
...,...,...,...
150,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
151,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
152,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...
153,/data/ik18445/projects/biorxiv-medrxiv-tdm/dat...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...,/data/ik18445/projects/biorxiv-medrxiv-tdm/out...


In [13]:
metadata_results_nested = []
for _ in meta_info["output_results_file"]:
    try:
        df = pd.read_csv(_)
        metadata_results_nested.append(df)
    except:
        continue
metadata_results = pd.concat(metadata_results_nested).reset_index(drop=True)
    
metadata_results.info()
metadata_results

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286789 entries, 0 to 286788
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   zip_path             286789 non-null  object 
 1   full_text_file       286789 non-null  object 
 2   doi                  286789 non-null  object 
 3   title                286788 non-null  object 
 4   version              286789 non-null  float64
 5   category             286782 non-null  object 
 6   year_month           286788 non-null  object 
 7   preprint_source      286789 non-null  object 
 8   publish_destination  22 non-null      object 
dtypes: float64(1), object(8)
memory usage: 19.7+ MB


,zip_path,full_text_file,doi,title,version,category,year_month,preprint_source,publish_destination
0,biorxiv/Back_Content/Batch_10/2b03c3ec-6c00-10...,content/083659.xml,10.1101/083659,Structural and Functional analysis of the GABA...,1.1,Biochemistry,"{'year': 2016, 'month': 10}",BIORXIV,NaN
1,biorxiv/Back_Content/Batch_10/be21e297-6bf6-10...,content/088120.xml,10.1101/088120,Electroceutical disinfection strategies impair...,1.1,Bioengineering,"{'year': 2016, 'month': 11}",BIORXIV,NaN
2,biorxiv/Back_Content/Batch_10/4362745b-6bf8-10...,content/090647.xml,10.1101/090647,"The emergence, evolution, and diversification ...",1.1,Evolutionary Biology,"{'year': 2016, 'month': 12}",BIORXIV,NaN
3,biorxiv/Back_Content/Batch_10/407b7537-6bfc-10...,content/054858.xml,10.1101/054858,NCS1 regulates Ca2+-Dependent Focal Exocytosis...,1.2,Cell Biology,"{'year': 2016, 'month': 11}",BIORXIV,NaN
4,biorxiv/Back_Content/Batch_10/e8854b54-6c05-10...,content/086934.xml,10.1101/086934,Spike interval coding of translatory optic flo...,1.1,Neuroscience,"{'year': 2016, 'month': 11}",BIORXIV,NaN
...,...,...,...,...,...,...,...,...,...
286784,medrxiv/Current_Content/March_2022/8eef593f-6e...,content/22272775.xml,10.1101/2022.03.22.22272775,Risk of death following SARS-CoV-2 infection o...,1.1,Epidemiology,"{'year': 2022, 'month': 3}",MEDRXIV,NaN
286785,medrxiv/Current_Content/March_2022/338749a2-6d...,content/22272553.xml,10.1101/2022.03.18.22272553,15-month follow-up of anti-spike receptor bind...,1.1,Infectious Diseases (except HIV/AIDS),"{'year': 2022, 'month': 3}",MEDRXIV,NaN
286786,medrxiv/Current_Content/March_2022/446c7d6f-7a...,content/22269512.xml,10.1101/2022.01.27.22269512,Investigation of air change rate and aerosol b...,1.2,Occupational and Environmental Health,"{'year': 2022, 'month': 3}",MEDRXIV,NaN
286787,medrxiv/Current_Content/March_2022/16ae3262-6e...,content/21257682.xml,10.1101/2021.05.23.21257682,Type 2 diabetes mellitus accelerates brain agi...,1.3,Endocrinology,"{'year': 2022, 'month': 3}",MEDRXIV,NaN


In [16]:
output_file = OUTPUT_DIR / "metadata_results.csv"
metadata_results.to_csv(output_file, index=False)

---

# Load

In [63]:
input_file = OUTPUT_DIR / "metadata_results.csv"
metadata_results = pd.read_csv(input_file)
metadata_results = metadata_results \
    .assign(
        year_month=lambda df: df["year_month"].apply(
            lambda e: ast.literal_eval(e) if not pd.isnull(e) else e
        )
    ) \
    .dropna(subset=["year_month"]) \
    .assign(
        time=lambda df: df["year_month"].apply(
            lambda e: "{year}-{month}".format(year=e["year"], month=e["month"]) 
        ).pipe(pd.to_datetime)
    )
metadata_results = metadata_results[
    metadata_results["preprint_source"].isin(["BIORXIV", "MEDRXIV"])
]

/tmp/ipykernel_41698/2768983954.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_results = pd.read_csv(input_file)


# Diagnostics

In [4]:
metadata_results[~metadata_results["publish_destination"].isna()]["publish_destination"]

1287         YNIMG
1434          PONE
1464         peerj
1578          BCAN
1675           mbc
1744         elife
1773        Genome
1791         elife
1983          GBIO
2287         elife
2545         elife
10322         pnas
10336        YNIMG
10426        elife
10799        YNIMG
10847        elife
18341          msb
22984    PCOMPBIOL
23865    PCOMPBIOL
46715         giga
47347    PMEDICINE
52249        YNIMG
Name: publish_destination, dtype: object

# Rough analysis

## total count

In [64]:
df_ts_total = metadata_results \
    [["time", "preprint_source"]].dropna() \
    .groupby(["time", "preprint_source"]).size() \
    .rename("count").to_frame().reset_index(drop=False)
df_ts_total

,time,preprint_source,count
0,2013-10-01,BIORXIV,5
1,2013-11-01,BIORXIV,64
2,2013-12-01,BIORXIV,69
3,2014-01-01,BIORXIV,71
4,2014-02-01,BIORXIV,85
...,...,...,...
154,2023-03-01,MEDRXIV,1270
155,2023-04-01,BIORXIV,4083
156,2023-04-01,MEDRXIV,968
157,2023-05-01,BIORXIV,4232


In [65]:
alt.Chart(df_ts_total).mark_bar() \
    .encode(
        x="time:T",
        y="count",
        color="preprint_source:N",
        tooltip=["time", "preprint_source", "count"]
    ) \
    .facet(row="preprint_source:N") \
    .interactive()

alt.FacetChart(...)

## categories

In [47]:
df_cat = metadata_results \
    [["category", "preprint_source"]].dropna() \
    .groupby(["category", "preprint_source"]).size() \
    .rename("count").to_frame().reset_index(drop=False) \
    .sort_values(["count"], ascending=False)
df_cat

,category,preprint_source,count
55,Neuroscience,BIORXIV,43608
8,Bioinformatics,BIORXIV,21623
50,Microbiology,BIORXIV,20597
29,Evolutionary Biology,BIORXIV,13852
36,Genomics,BIORXIV,12642
...,...,...,...
75,Psychiatry and Clinical Psychology,BIORXIV,1
11,CELL BIOLOGY,BIORXIV,1
10,CANCER BIOLOGY,BIORXIV,1
22,EPIDEMIOLOGY,BIORXIV,1


In [50]:
alt.Chart(df_cat).mark_bar() \
    .encode(
        y="category:N",
        x="count:Q",
        tooltip=["category", "count"]
    ) \
    .properties(
        width=400,
        height=800
    ) \
    .facet(
        row="preprint_source:N"
    ) \
    .interactive()

alt.FacetChart(...)

## TS distribution of categories

In [66]:
df_ts_cat = metadata_results \
    [["time", "category", "preprint_source"]].dropna()

df_ts_cat

,time,category,preprint_source
0,2016-10-01,Biochemistry,BIORXIV
1,2016-11-01,Bioengineering,BIORXIV
2,2016-12-01,Evolutionary Biology,BIORXIV
3,2016-11-01,Cell Biology,BIORXIV
4,2016-11-01,Neuroscience,BIORXIV
...,...,...,...
286784,2022-03-01,Epidemiology,MEDRXIV
286785,2022-03-01,Infectious Diseases (except HIV/AIDS),MEDRXIV
286786,2022-03-01,Occupational and Environmental Health,MEDRXIV
286787,2022-03-01,Endocrinology,MEDRXIV


In [67]:
df_ts_cat_sum = df_ts_cat \
    .groupby(["time", "category", "preprint_source"]).size() \
    .rename("count").to_frame().reset_index(drop=False)
df_ts_cat_sum

,time,category,preprint_source,count
0,2013-10-01,Cancer Biology,BIORXIV,1
1,2013-10-01,Evolutionary Biology,BIORXIV,1
2,2013-10-01,Genetics,BIORXIV,1
3,2013-10-01,Genomics,BIORXIV,1
4,2013-10-01,Synthetic Biology,BIORXIV,1
...,...,...,...,...
4823,2023-05-01,Systems Biology,BIORXIV,106
4824,2023-05-01,Toxicology,MEDRXIV,1
4825,2023-05-01,Transplantation,MEDRXIV,5
4826,2023-05-01,Urology,MEDRXIV,5


In [69]:
alt.Chart(df_ts_cat_sum).mark_bar() \
    .encode(
        x="time:T",
        y="count:Q",
        color="category:N",
        tooltip=["time", "count", "category"]
    ) \
    .properties(
        width=800,
        height=400
    ) \
    .facet(
        row="preprint_source:N"
    ) \
    .interactive()

alt.FacetChart(...)